In [ ]:
# -*- coding: utf-8 -*-
"""
.. module:: skimpy
   :platform: Unix, Windows
   :synopsis: Simple Kinetic Models in Python

.. moduleauthor:: SKiMPy team

[---------]

Copyright 2017 Laboratory of Computational Systems Biotechnology (LCSB),
Ecole Polytechnique Federale de Lausanne (EPFL), Switzerland

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIE CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

"""
from pytfa.io.json import load_json_model
from skimpy.io.yaml import load_yaml_model
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, \
    load_concentrations, load_equilibrium_constants
from skimpy.sampling.simple_parameter_sampler import SimpleParameterSampler
from skimpy.core import *
from skimpy.mechanisms import *
from scipy.linalg import eigvals as eigenvalues
from sympy import Symbol
from skimpy.core.parameters import ParameterValues,ParameterValuePopulation
import pandas as pd
import numpy as np
import pickle

import helper as hp
from skimpy.core.parameters import load_parameter_population

#sys.path.append("..")


#Convenience factors
NCPU = 32
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING


exp_id = 'fdp1'

met_model = 'ecoli_shikki_regulation'
# names_km = hp.load_pkl(f'models/{met_model}/parameter_names_km_fdp1.pkl')
#ss_idx = 1712

base = f"../models/{met_model}"
path_to_tmodel = f'{base}/thermo/varma_{exp_id}'
path_to_kmodel = f'{base}/kinetic/kin_varma_curated.yaml'



In [ ]:

tmodel = load_json_model(path_to_tmodel)
kmodel = load_yaml_model(path_to_kmodel)
# NEW STUFF #
from skimpy.io.regulation import load_enzyme_regulation
# Add regulation data to kinetic model
path_to_regulation_data = f'{base}/allosteric_regulations.csv'
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel.reactants.keys()))]
kmodel = load_enzyme_regulation(kmodel, df_regulations_all)
#-----------#
kmodel.prepare()
kmodel.compile_jacobian(sim_type=QSSA, ncpu=NCPU)

In [ ]:
path_to_samples = f'{base}/steady_state_samples/steady_states.csv'

ss_idx = 1712 #[1715,1717,2392,2482,3927,4230,4241,4456, 4468]
samples = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]

# Load ss fluxes and concentrations
flux_series = load_fluxes(samples, tmodel, kmodel,
                  density=DENSITY,
                  ratio_gdw_gww=GDW_GWW_RATIO,
                  concentration_scaling=CONCENTRATION_SCALING,
                  time_scaling=TIME_SCALING)

conc_series = load_concentrations(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING)


fluxes = [flux_series[this_reaction.name] for this_reaction in
                  kmodel.reactions.values()]
concentrations = np.array([conc_series[this_variable] for
                                   this_variable in
                                   kmodel.variables.keys()])
        
# Fetch equilibrium constants
k_eq = load_equilibrium_constants(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING,
                          in_place=True)

symbolic_concentrations_dict = {Symbol(k): v for k, v in conc_series.items()}

sampling_parameters = SimpleParameterSampler.Parameters(n_samples=1)
sampler = SimpleParameterSampler(sampling_parameters)

sampler._compile_sampling_functions(kmodel, symbolic_concentrations_dict, [])
model_param = kmodel.parameters
param_population = load_parameter_population(f'{base}/parameters/kinetic_params_top_10_models.hdf5')

In [ ]:
kmodel.parameters['amob_c']

In [ ]:
param_population['1712,6']['amob_c']

In [ ]:
param_pop = []
max_eig = []
min_eig = []
all_eigs = []
samples_to_simulate = list(param_population._index.keys())
for ix in samples_to_simulate[0:1]:
    parameter_sample = param_population[ix]
    tfa_id = int(ix.split(',')[0])
    samples = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[tfa_id, 0:]

    # Load ss fluxes and concentrations
    flux_series = load_fluxes(samples, tmodel, kmodel,
                      density=DENSITY,
                      ratio_gdw_gww=GDW_GWW_RATIO,
                      concentration_scaling=CONCENTRATION_SCALING,
                      time_scaling=TIME_SCALING)

    conc_series = load_concentrations(samples, tmodel, kmodel,
                              concentration_scaling=CONCENTRATION_SCALING)


    fluxes = [flux_series[this_reaction.name] for this_reaction in
                      kmodel.reactions.values()]
    concentrations = np.array([conc_series[this_variable] for
                                       this_variable in
                                       kmodel.variables.keys()])

    # Fetch equilibrium constants
    k_eq = load_equilibrium_constants(samples, tmodel, kmodel,
                              concentration_scaling=CONCENTRATION_SCALING,
                              in_place=True)

    symbolic_concentrations_dict = {Symbol(k): v for k, v in conc_series.items()}
    
    this_jacobian = kmodel.jacobian_fun(fluxes,
                                        concentrations, parameter_sample)

    this_real_eigenvalues = sorted(np.real(eigenvalues(this_jacobian.todense())))
    max_eig.append(this_real_eigenvalues[-1])
    min_eig.append(this_real_eigenvalues[0])
    all_eigs.append(this_real_eigenvalues)

In [ ]:
fluxes

In [ ]:
concentrations

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Load HDF5 file
with h5py.File(f"{base}/parameters/kinetic_params_top_10_models.hdf5", 'r') as file:
    parameter_names = file['parameter_names'][:]
    num_sets = int(file['num_parameters_sets'][()])
    parameter_sets = [file[f'parameter_set_{i}'][:] for i in range(num_sets)]

In [ ]:
# Convert parameter names to a list of strings if necessary
parameter_names = [name.decode() if isinstance(name, bytes) else name for name in parameter_names]

# Sort parameter names and corresponding values
sorted_indices = np.argsort(parameter_names)
sorted_parameter_names = np.array(parameter_names)[sorted_indices]

# Create a sorted matrix
sorted_matrix = np.array(parameter_sets)[:, sorted_indices].T


In [ ]:
# Filter parameters based on the specified substrings
filtered_indices = [i for i, name in enumerate(sorted_parameter_names) if "activation" in name or "inhibition" in name or "km_" in name]

# Randomly select 40 indices from the filtered list (if there are more than 40)
if len(filtered_indices) > 40:
    selected_indices = np.random.choice(filtered_indices, 40, replace=False)
else:
    selected_indices = filtered_indices

# Extract the selected parameter names and their values from parameter_set_0
selected_parameter_names = sorted_parameter_names[selected_indices]
selected_values = sorted_matrix[selected_indices, 0]

# Plotting the selected parameters and their values
plt.figure(figsize=(10, 6))
plt.plot(selected_parameter_names, selected_values, marker='o', linestyle='-', color='b')
plt.title('Values from parameter_set_0 for selected parameters')
plt.xlabel('Parameter Names')
plt.ylabel('Values')
plt.yscale('log')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to not cut off labels
plt.show()


In [ ]:
# Extract the selected parameter names and their values from parameter_set_0
selected_parameter_names = sorted_parameter_names[filtered_indices]
selected_values = sorted_matrix[filtered_indices, 0]

selected_values = selected_values/1e9
selected_values = np.log(selected_values)
# Plotting the selected parameters and their values
plt.figure(figsize=(50, 6))
plt.plot(selected_parameter_names, selected_values, marker='o', linestyle='-', color='b')
plt.title('Values from parameter_set_0 for selected parameters')
plt.xlabel('Parameter Names')
plt.ylabel('Values')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to not cut off labels
plt.show()

In [ ]:
# Initialize a figure
plt.figure(figsize=(50, 8))

# Create a dataset for boxplot where each row corresponds to a parameter, and each column corresponds to a different set
data_for_boxplot = [np.array(parameter_sets)[:, sorted_indices[i]] for i in filtered_indices]

# Create the boxplot
plt.boxplot(data_for_boxplot, labels=selected_parameter_names)

# Add plot details
plt.xticks(rotation=90)
plt.xlabel('Parameter Names')
plt.ylabel('Values')
plt.yscale('log')
plt.title('Distribution of Parameter Values Across Sets')
plt.tight_layout()  # Adjust layout to make sure everything fits without overlap
plt.savefig('all_parameter_sets')